In [ ]:
# %%
# @title 📦 Install Required Libraries
%pip install indeterminatebeam matplotlib ipywidgets


In [ ]:


# %%
# @title 📁 Imports and Setup
import os
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from indeterminatebeam import Beam, PointLoadV, DistributedLoadV, PointTorque
from indeterminatebeam.supports import Pin as PinSupport, Roller as RollerSupport, Fixed as FixedSupport

support_map = {
    "pin": PinSupport,
    "roller": RollerSupport,
    "fixed": FixedSupport
}

# @title 🏗️ Section Library Setup
library_folder = "SteelLibrary/Members/Main"
csv_files = [f for f in os.listdir(library_folder) if f.endswith(".csv")]
section_types = sorted(set(f[:3] for f in csv_files))

section_type_dropdown = widgets.Dropdown(
    options=section_types, description="Section Type:", layout=widgets.Layout(width="250px")
)
series_dropdown = widgets.Dropdown(options=[], description="Series:", layout=widgets.Layout(width="250px"))
member_dropdown = widgets.Dropdown(options=[], description="Member:", layout=widgets.Layout(width="350px"))
material_output = widgets.Output()
geometry_output = widgets.Output()
current_df = pd.DataFrame()

section_props = {}

# Dropdown behavior
def get_grade_from_series(series_name):
    try:
        return int(''.join(filter(str.isdigit, series_name))) * 10
    except:
        return "Unknown"

def update_series(change):
    section_type = change['new']
    options = sorted([f.replace(".csv", "") for f in csv_files if f.startswith(section_type)])
    series_dropdown.options = options
    member_dropdown.options = []
    material_output.clear_output()
    geometry_output.clear_output()

def update_member_list(change):
    global current_df
    full_path = os.path.join(library_folder, f"{change['new']}.csv")
    try:
        df = pd.read_csv(full_path, skiprows=[1])
        current_df = df
        member_dropdown.options = df["Description"].dropna().tolist()
    except Exception as e:
        current_df = pd.DataFrame()
        member_dropdown.options = []
        with material_output:
            print(f"❌ Error reading file: {e}")

def display_properties(change):
    material_output.clear_output()
    geometry_output.clear_output()
    member = change['new']
    if current_df.empty or member not in current_df["Description"].values:
        return
    row = current_df[current_df["Description"] == member].iloc[0]
    series = series_dropdown.value
    grade = get_grade_from_series(series)
    E = 200_000
    Ix = row.get("Ix", "N/A")
    section_props['E'] = E * 1e6
    section_props['Ix'] = float(Ix) * 1e-12
    with material_output:
        print(f"✅ Selected: {member}")
        print(f"🟦 Grade: {grade}")
        print(f"📐 E: {E} MPa")
        print(f"📘 Ix: {Ix} ×10⁶ mm⁴")
    with geometry_output:
        display(row.drop(labels=["Description", "Type"], errors='ignore').to_frame(name='Value'))

section_type_dropdown.observe(update_series, names="value")
series_dropdown.observe(update_member_list, names="value")
member_dropdown.observe(display_properties, names="value")

# Display Section UI
section_ui = widgets.VBox([
    section_type_dropdown,
    series_dropdown,
    member_dropdown,
    widgets.HTML("<b>Material Properties</b>"),
    material_output,
    widgets.HTML("<b>Geometric Properties</b>"),
    geometry_output
])
display(section_ui)
update_series({'new': section_type_dropdown.value})


# %%
# @title 🧱 Beam Definition Inputs
num_spans_widget = widgets.BoundedIntText(value=2, min=1, max=6, description='Spans:')
generate_inputs_btn = widgets.Button(description='Generate Inputs', button_style='info')

span_inputs_box = widgets.VBox()
udl_inputs_box = widgets.VBox()
vdl_inputs_box = widgets.VBox()
point_loads_box = widgets.VBox()
moment_loads_box = widgets.VBox()
solve_output = widgets.Output()

support_types = ["pin", "roller", "fixed"]

# Dynamic input generator
def generate_span_inputs(_):
    span_widgets, udl_widgets, vdl_widgets = [], [], []
    for i in range(num_spans_widget.value):
        length = widgets.FloatText(value=6.0, description=f'L{i+1} (m)', layout=widgets.Layout(width='150px'))
        udl = widgets.FloatText(value=5.0, description=f'UDL{i+1} (kN/m)', layout=widgets.Layout(width='200px'))
        vdl_start = widgets.FloatText(value=0.0, description=f'VDL{i+1} Start (kN/m)', layout=widgets.Layout(width='220px'))
        vdl_end = widgets.FloatText(value=0.0, description='End (kN/m)', layout=widgets.Layout(width='160px'))
        left_support = widgets.Dropdown(options=support_types, description=f'Left{i+1}', layout=widgets.Layout(width='180px'))
        span_widgets.append(widgets.HBox([length, left_support]))
        udl_widgets.append(udl)
        vdl_widgets.append(widgets.HBox([vdl_start, vdl_end]))
    right_support = widgets.Dropdown(options=support_types, description=f'Right', layout=widgets.Layout(width='180px'))
    span_widgets.append(widgets.HBox([widgets.Label(value=""), right_support]))
    span_inputs_box.children = span_widgets
    udl_inputs_box.children = udl_widgets
    vdl_inputs_box.children = vdl_widgets

    # Point & moment load UI
    point_widgets, moment_widgets = [], []
    for i in range(3):
        loc = widgets.FloatText(value=2.0, description=f'P{i+1} Loc (m)', layout=widgets.Layout(width='180px'))
        mag = widgets.FloatText(value=-10.0, description='Mag (kN)', layout=widgets.Layout(width='150px'))
        point_widgets.append(widgets.HBox([loc, mag]))
        mloc = widgets.FloatText(value=3.0, description=f'M{i+1} Loc (m)', layout=widgets.Layout(width='180px'))
        mmag = widgets.FloatText(value=5.0, description='Mag (kNm)', layout=widgets.Layout(width='150px'))
        moment_widgets.append(widgets.HBox([mloc, mmag]))
    point_loads_box.children = point_widgets
    moment_loads_box.children = moment_widgets

generate_inputs_btn.on_click(generate_span_inputs)

display(widgets.VBox([
    num_spans_widget,
    generate_inputs_btn,
    widgets.HTML("<b>Spans & Supports</b>"),
    span_inputs_box,
    widgets.HTML("<b>UDLs</b>"),
    udl_inputs_box,
    widgets.HTML("<b>VDLs</b>"),
    vdl_inputs_box,
    widgets.HTML("<b>Point Loads</b>"),
    point_loads_box,
    widgets.HTML("<b>Moments</b>"),
    moment_loads_box,
    solve_output
]))

from indeterminatebeam import Beam, PointLoadV, DistributedLoadV

from indeterminatebeam.supports import Pin, Roller, Fixed

support_map = {
    "pin": Pin,
    "roller": Roller,
    "fixed": Fixed
}

solve_output.clear_output()
# %%
# @title 🔍 Solve Beam
solve_btn = widgets.Button(description='Solve Beam', button_style='success')

def solve_beam(_):
    solve_output.clear_output(wait=True)
    try:
        E = section_props.get("E", 200e9)
        I = section_props.get("Ix", 1e-6)

        lengths = []
        support_types_input = []
        for row in span_inputs_box.children[:-1]:
            l, s = row.children
            lengths.append(float(l.value))
            support_types_input.append(s.value)
        support_types_input.append(span_inputs_box.children[-1].children[1].value)

        total_length = sum(lengths)
        beam = Beam(span=total_length, E=E, I=I)

        # Add supports
        pos = 0
        beam.add_supports(support_map[support_types_input[0]](pos))
        for i in range(1, len(lengths)):
            pos += lengths[i - 1]
            beam.add_supports(support_map[support_types_input[i]](pos))
        pos += lengths[-1]
        beam.add_supports(support_map[support_types_input[-1]](pos))

        # Add span loads
        pos = 0
        for i, L in enumerate(lengths):
            w = float(udl_inputs_box.children[i].value) * 1e3
            if abs(w) > 1e-3:
                beam.add_loads(DistributedLoadV(w, (pos, pos + L)))
            vdl = vdl_inputs_box.children[i].children
            v1 = float(vdl[0].value) * 1e3
            v2 = float(vdl[1].value) * 1e3
            if abs(v1) > 1e-3 or abs(v2) > 1e-3:
                beam.add_loads(DistributedLoadV((v1, v2), (pos, pos + L)))
            pos += L

        # Point & moment loads
        for row in point_loads_box.children:
            loc = float(row.children[0].value)
            mag = float(row.children[1].value) * 1e3
            beam.add_loads(PointLoadV(mag, loc))

        for row in moment_loads_box.children:
            loc = float(row.children[0].value)
            mag = float(row.children[1].value) * 1e3
            beam.add_loads(PointTorque(mag, loc))

        beam.analyse()

        with solve_output:
            clear_output(wait=True)
            print("\n📌 Reactions:")
            for k, v in beam.get_reactions().items():
                print(f"{k}: {v:.2f} N")

            print("\n📈 Max Effects:")
            print(f"• Max Moment: {beam.max_moment():.2f} Nm")
            print(f"• Max Shear: {beam.max_shear():.2f} N")
            print(f"• Max Deflection: {beam.max_deflection():.4f} m")

            fig, axs = plt.subplots(4, 1, figsize=(12, 16))
            beam.plot_beam_external(ax=axs[0])
            axs[0].set_title("Loading Diagram")
            beam.plot_shear_force(ax=axs[1])
            axs[1].set_title("Shear Force Diagram")
            beam.plot_bending_moment(ax=axs[2])
            axs[2].set_title("Bending Moment Diagram")
            beam.plot_deflection(ax=axs[3])
            axs[3].set_title("Deflection Diagram")
            plt.tight_layout()
            plt.show()

            summary = pd.DataFrame({
                "Effect": ["Max Moment", "Max Shear", "Max Deflection"],
                "Value": [
                    f"{beam.max_moment():.2f} Nm",
                    f"{beam.max_shear():.2f} N",
                    f"{beam.max_deflection():.4f} m"
                ]
            })
            display(summary)

    except Exception as e:
        with solve_output:
            print(f"❌ Error: {e}")

display(solve_btn)
solve_btn.on_click(solve_beam)
